In [ ]:
#@title Load repo (if needed)
!git clone https://github.com/JoePenna/Dreambooth-Stable-Diffusion
%cd Dreambooth-Stable-Diffusion

In [ ]:
#@title BUILD ENV
!pip install numpy==1.23.1
!pip install pytorch-lightning==1.7.6
!pip install csv-logger
!pip install torchmetrics==0.11.1
!pip install torch-fidelity==0.3.0
!pip install albumentations==1.1.0
!pip install opencv-python==4.7.0.72
!pip install pudb==2019.2
!pip install omegaconf==2.1.1
!pip install pillow==9.4.0
!pip install einops==0.4.1
!pip install transformers==4.25.1
!pip install kornia==0.6.7
!pip install diffusers[training]==0.3.0
!pip install captionizer==1.0.1
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install -e .
!pip install huggingface_hub
!pip install gitpython

print("The instance needs to restart to apply changes.")

import os
os._exit(00)

In [ ]:
#@title # Required - Navigate back to the directory.
%cd Dreambooth-Stable-Diffusion

In [ ]:
#@markdown Download the 1.5 sd model with the improved vae
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="panopstor/EveryDream",
 filename="sd_v1-5_vae.ckpt"
)

# Move the sd_v1-5_vae.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")


In [2]:
#@title # Download Regularization Images
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0 <br />

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

In [ ]:
#@title # Training Images
#@markdown ## Upload your training images
#@markdown WARNING: Be sure to upload an even amount of images, otherwise the training inexplicably stops at 1500 steps. <br />
#@markdown - 2-3 full body
#@markdown - 3-5 upper body
#@markdown - 5-12 close-up on face  <br /> <br />
#@markdown The images should be as close as possible to the kind of images you’re trying to make (most of the time, that means no selfies).
from google.colab import files
from IPython.display import clear_output

# Create the directory
!rm -rf training_images
!mkdir -p training_images

# Upload the files
uploaded = files.upload()
for filename in uploaded.keys():
 updated_file_name = filename.replace(" ", "_")
 !mv "{filename}" "training_images/{updated_file_name}"
 clear_output()

# Tell the user what is going on
training_images_file_paths = !find training_images/*
if len(training_images_file_paths) == 0:
 print("❌ no training images found. Please upload images to training_images")
else:
 print("✅ " + str(len(training_images_file_paths)) + " training images found")


In [ ]:
#@title # Training

#@markdown This isn't used for training, just to help you remember what your trained into the model.
project_name = "project_name" #@param {type:"string"}

# MAX STEPS
#@markdown How many steps do you want to train for?
max_training_steps = 2000 #@param {type:"integer"}

#@markdown Match class_word to the category of the regularization images you chose above.
class_word = "person" #@param ["man", "person", "woman"] {allow-input: true}

#@markdown Would you like to save a model every X steps? (Example: 250 would output a trained model at 250, 500, 750 steps, etc)
save_every_x_steps = -1 #param {type:"integer"}

#@markdown If you are training a person's face, set this to True
i_am_training_a_persons_face = False #@param {type:"boolean"}

flip_p_arg = 0.0 if i_am_training_a_persons_face else 0.5


#@markdown This is the unique token you are incorporating into the stable diffusion model.
token = "firstNameLastName" #@param {type:"string"}

# 0 Saves the checkpoint when max_training_steps is reached.
# 250 saves the checkpoint every 250 steps as well as when max_training_steps is reached.
save_every_x_steps = 0

reg_data_root = "/content/Dreambooth-Stable-Diffusion/regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --project_name "{project_name}" \
 --debug False \
 --max_training_steps {max_training_steps} \
 --token "{token}" \
 --training_model "model.ckpt" \
 --training_images "/content/Dreambooth-Stable-Diffusion/training_images" \
 --regularization_images "{reg_data_root}" \
 --class_word "{class_word}" \
 --flip_p {flip_p_arg} \
 --save_every_x_steps {save_every_x_steps}

In [ ]:
#@title Save model in google drive
from google.colab import drive
drive.mount('/content/drive')

!cp trained_models/{file_name.ckpt} /content/drive/MyDrive/{file_name.ckpt}